In [1]:
import sys
import random
import logging
import numpy as np 

sys.path.append("../interfaces/")
sys.path.append("../db_functions/")
sys.path.append("../../services/")

from unittest.mock import patch
import datetime

from connection import connect_to_db
from ParkingSpaceInterface import ParkingSpaceInterface as PSI
import service_functions as serv_func
from gen_test_data import reset_all_ps 

In [2]:
connect_to_db()

True

In [3]:
def park_car_at_time(space_id, car_id, time):
    @patch("parking_space_functions.now")
    def park(mock_now):
        mock_now.return_value = time
        return serv_func.park_car(space_id, car_id)

    return park()


def leave_car_at_time(car_id, time):
    @patch("parking_space_functions.now")
    def leave(mock_now):
        mock_now.return_value = time
        return serv_func.leave_car(car_id)

    return leave()


def generate_license_plate():
    import string

    letters = "".join(random.choices(string.ascii_uppercase, k=3))
    numbers = "".join(random.choices(string.digits, k=4))
    return letters + numbers

In [4]:
def gen_ps_history():
    logging.basicConfig(level=logging.DEBUG)

    park_cnt = {}  # 停車車次
    flow = {}

    # 清空所有停車格
    logging.debug("reset_all_ps")
    reset_all_ps()
    logging.debug("reset_all_ps done")

    # 設定產生資料的時間範圍
    start_date = datetime.datetime(2023, 11, 1)
    end_date = datetime.datetime(2023, 11, 1)

    # 產生車牌號碼
    car_ids = [generate_license_plate() for i in range(3000)]
    car_ids = list(set(car_ids))

    # 取得所有停車格的 id
    pss = PSI.read_all_ps()
    spaces_ids = [ps["space_id"] for ps in pss]

    # 紀錄每台車停在哪個停車格
    park_at = {}

    # probability
    probability = {
        "park": np.array([1, 1, 1, 1, 1, 5, 8, 15, 15, 15, 7, 3, 5, 4, 3, 2, 2, 2, 2, 2, 2, 1, 1, 1]),
        "leave": np.array([1, 1, 1, 1, 1, 3, 2, 2, 2, 2, 7, 16, 4, 4, 1, 4, 5, 6, 8, 8, 9, 5, 5, 2]),
    }

    probability["park"] = probability["park"] / 50
    probability["leave"] = probability["leave"] / 50

    # 開始模擬
    current_date = start_date
    while current_date <= end_date:
        park_cnt[current_date] = 0

        for hour in range(24):
            current_hour = current_date + datetime.timedelta(hours=hour)
            flow[current_hour] = len(park_at)

            for sec in range(0, 3600, 5):
                random_num = random.random()
                park_prob = probability["park"][hour]
                leave_prob = probability["leave"][hour] + park_prob

                if random_num < park_prob:
                    park_cnt[current_date] += 1

                    # 選出一個沒有停車的車子和一個沒有被佔用的停車格
                    not_parked_car_ids = [car_id for car_id in car_ids if car_id not in park_at.keys()]
                    not_occupied_space_ids = [space_id for space_id in spaces_ids if space_id not in park_at.values()]
                    if len(not_occupied_space_ids) == 0 or len(not_parked_car_ids) == 0:
                        continue
                    car_id = random.choice(not_parked_car_ids)
                    space_id = random.choice(not_occupied_space_ids)

                    # 停車
                    current_time = current_hour + datetime.timedelta(seconds=sec)
                    park_at[car_id] = space_id
                    success, msg = park_car_at_time(space_id, car_id, current_time)

                    # 印出停車資訊
                    logging.debug(f"{current_time}: {msg}")

                elif random_num < leave_prob:
                    # 選出一個已經停車的車子
                    parking_car_ids = [car_id for car_id in car_ids if car_id in park_at.keys()]
                    if len(parking_car_ids) == 0:
                        continue
                    car_id = random.choice(parking_car_ids)

                    # 車子離場
                    current_time = current_hour + datetime.timedelta(seconds=sec)
                    park_at.pop(car_id)
                    success, msg = leave_car_at_time(car_id, current_time)

                    # 印出離場資訊
                    logging.debug(f"{current_time}: {msg}")

        current_date += datetime.timedelta(days=1)

    return park_cnt, park_at, flow

In [5]:
gen_ps_history()

DEBUG:root:reset_all_ps
DEBUG:root:reset_all_ps done
DEBUG:root:2023-11-01 00:10:40: 車輛 UHW2426 已停入停車位 5063
DEBUG:root:2023-11-01 00:19:05: 車輛 UHW2426 已離場，共停了 0:08:25
DEBUG:root:2023-11-01 00:22:25: 車輛 SPW2046 已停入停車位 2079
DEBUG:root:2023-11-01 00:25:15: 車輛 SPW2046 已離場，共停了 0:02:50
DEBUG:root:2023-11-01 00:26:50: 車輛 BJY5461 已停入停車位 1115
DEBUG:root:2023-11-01 00:30:55: 車輛 BJY5461 已離場，共停了 0:04:05
DEBUG:root:2023-11-01 00:33:50: 車輛 RCB9914 已停入停車位 2022
DEBUG:root:2023-11-01 00:36:00: 車輛 BIM3755 已停入停車位 3048
DEBUG:root:2023-11-01 00:36:45: 車輛 BIM3755 已離場，共停了 0:00:45
DEBUG:root:2023-11-01 00:37:00: 車輛 XUP4516 已停入停車位 5007
DEBUG:root:2023-11-01 00:37:25: 車輛 RCB9914 已離場，共停了 0:03:35
DEBUG:root:2023-11-01 00:42:30: 車輛 XUP4516 已離場，共停了 0:05:30
DEBUG:root:2023-11-01 00:43:55: 車輛 WYR7822 已停入停車位 3069
DEBUG:root:2023-11-01 00:51:20: 車輛 IZS3756 已停入停車位 1118
DEBUG:root:2023-11-01 00:51:25: 車輛 XRI2409 已停入停車位 5005
DEBUG:root:2023-11-01 00:52:00: 車輛 WYR7822 已離場，共停了 0:08:05
DEBUG:root:2023-11-01 00:54:00: 車輛 FFJ1

({datetime.datetime(2023, 11, 1, 0, 0): 1475},
 {'SYV4241': '1158'},
 {datetime.datetime(2023, 11, 1, 0, 0): 0,
  datetime.datetime(2023, 11, 1, 1, 0): 4,
  datetime.datetime(2023, 11, 1, 2, 0): 5,
  datetime.datetime(2023, 11, 1, 3, 0): 13,
  datetime.datetime(2023, 11, 1, 4, 0): 16,
  datetime.datetime(2023, 11, 1, 5, 0): 16,
  datetime.datetime(2023, 11, 1, 6, 0): 38,
  datetime.datetime(2023, 11, 1, 7, 0): 110,
  datetime.datetime(2023, 11, 1, 8, 0): 337,
  datetime.datetime(2023, 11, 1, 9, 0): 513,
  datetime.datetime(2023, 11, 1, 10, 0): 660,
  datetime.datetime(2023, 11, 1, 11, 0): 652,
  datetime.datetime(2023, 11, 1, 12, 0): 471,
  datetime.datetime(2023, 11, 1, 13, 0): 477,
  datetime.datetime(2023, 11, 1, 14, 0): 481,
  datetime.datetime(2023, 11, 1, 15, 0): 505,
  datetime.datetime(2023, 11, 1, 16, 0): 488,
  datetime.datetime(2023, 11, 1, 17, 0): 418,
  datetime.datetime(2023, 11, 1, 18, 0): 358,
  datetime.datetime(2023, 11, 1, 19, 0): 264,
  datetime.datetime(2023, 11, 1